# PS 88 Project Part 2: Replication (Sheve and Stasavage)
**General instructions:** these replications will be similar to labs, though with a bit less guidance than we would typicallt give. An advantage you have is that the answer you are supposed to get for most questions is in the paper.

When we say to "replicate" a table we don't mean you need to reproduce every bit of formatting. Typically for regressions we just want you to verify that the coefficients match those reported in the table. For graphs, the output should look pretty similar, though again you don't need to make the formatting look exactly the same. 


On to the paper! In this notebook we will be exploring the replication data from Mass Warfare and the Demand for Progressive Taxation* by Kenneth Scheve and David Stasavage. The original paper is linked [here](https://isps.yale.edu/sites/default/files/publication/2012/12/ISPS10-042.pdf). 


Let's start by importing the packages we'll be using in this notebook:

In [ ]:
# Run this cell to import the packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
%matplotlib inline
plt.rcParams['figure.figsize'] = (16,8)
plt.rcParams['figure.dpi'] = 150
sns.set()
pd.set_option('display.max_columns', None)

Next, run the cell below to read in the data we will be using:

In [ ]:
data_string = "data/Scheve_Stasavage_IO_2010_CoWreplicationdata.csv"
ss = pd.read_csv(data_string)
ss

A nice way to explore a data frame is with the `.info()` function, which tells how many rows/columns there are, and the names of the columns.

In [ ]:
ss.info()

This data frame is in "country-year" format, meaning each entry corresponds to one year in one country (E.g., France in 1958).

To get a better sense of the data, let's explore some individual variables. We can use the `.value_counts()` function on the `year` variable to see how many data points there are for each year:

In [ ]:
ss['year'].value_counts()

So there are 8 observations for the years which we can display.

**Question 1.1. Use the `.value_counts()` function on the `country` variable to see what countries are in the data frame and how many years of data there are for each.**

In [ ]:
# Code for 1.1

Our key dependent variable is the tax rate on the highest earners, called `topratep`. 

**Question 1.2. Make a histogram of `topratep`.**

In [ ]:
# Code for 1.2

A key independent variable will be an indicator for whether the country is one that participated in WWI, and the year is after mobilization for the war. This variable is called `wwihighmobaft`. Let's plot this as a function of year.

In [ ]:
sns.scatterplot(x='year', y='wwihighmobaft', data=ss)

All country-years are coded as 0 before 1915 or so, and then there are some 0s and 1s. Which makes sense!

To see which countries sometimes eventually mobilized we can use the `crosstab()` function, which is similar to a pivot table. 

To see an example, a variable we will use later is `himobpopyear2p`, which is equal to 1 in years where the country had at least 2% of the population mobilized for war. 

In [ ]:
pd.crosstab(ss['country'], ss['himobpopyear2p'])  

**Question 1.3. Make a crosstab of the `country` and `wwihighmobaft` variables. What can we learn from this table?**

In [ ]:
# Code for 1.3

*Words for 1.3*

## Part 2. Graphing the Tax Rate of Participant and Non-Participant Countries

In this section, we'll be looking at the tax rates of countries that participated and did not participate in WWI and how they differ.


We'll start by splitting our dataframe into two: one with countries that participated in WWI and one with countries that did not.

We can learn from the previous question which countries participated. A nice way to restrict the data to these countries is to store their names as an array, and then is the `isin` function to identify rows in the data frame where `country` matches one of the names.

In [ ]:
part_list = ["UK", "France", "USA", "Canada"]
part = ss[ss['country'].isin(part_list)]
part

**Question 2.1. Make a dataframe called `nonpart` with the countries which did not participate in WWI (see your answer to 1.3 or the paper to identify these).**

In [ ]:
# Code for 2.1

To replicate figure 1, we also want to restrict to the years 1900 to 1930. We'll show you how to do this for the participant data frame:

In [ ]:
part_close = part[(part["year"]>=1900)  & (part["year"]<=1930)]

**Question 2.2 Create a data frame called `non_part_close` which subsets `non_part` to the observations where `year` is greater than or equal to 1900 and less than or equal to 1930.** 

In [ ]:
# Code for 2.2

Now that we have selected the correct time period for our analysis, let's visualize the trends between participant and non-participant countries.

In order to accomplish this comparison, we will be using two calls to `sns.lineplot`. We will also be adding in a descriptive title and axis labels. 

Since we haven't used this much in class, here is an overview of the main `sns.lineplot` arguments:
 * `x`: The name of the column for the x axis
 * `y`: The name of the column for the y axis
 * `data`: The name of the data frame that the visualization will be using
 * `label`: The name of what the lineplot will be called
 * `ci`: Whether the lineplot should have a confidence interval surrounding it. Set this argument to `False` to remove confidence intervals.

To give an example, let's plot the average GDP per capita and top tax rate across the entire sample.

In [ ]:
sns.lineplot(x="year",y="gdppcp",data=ss,label="GDP per Capita",ci=False)
sns.lineplot(x="year",y="topratep",data=ss, label="Top Tax Rate", ci=False)
plt.xlabel("Year")
plt.ylabel("GDP per Capita and Top Tax Rates")
plt.title("Long Run Economic Indicators")

**Question 2.3. We are now ready to replicate figure 1 from the paper. To do this, you should use `sns.lineplot` twice, with `year` as the x axis and `topratep` as the y axis, first for the participants and second for the non-participants. The important thing is to get the lines right, but you should label the axes and the plot in an informative way as well.**

In [ ]:
# Code for 2.3

**Question 2.4. Now make a similar graph which does not restrict from 1900-1930 (i.e., use the `part` and `nonpart` dataframes.**

In [ ]:
# Code for 2.4

**Question 2.5. What do these graphs suggest about the effect of participating in WW I on top income tax rates?**

*Words for 2.5*

## Part 3: Regressions

Now let's replicate some columns from Table 1 of the paper. Each column corresponds to a multivariate regression with `topratep` as the dependent variable, on the sample of all 8 countries from 1900 to 1930. Let's make a data frame with the appropriate observations.

In [ ]:
all_close = ss[(ss['year'] >= 1900) & (ss['year']<= 1930)]

The regression in the first column includes `wwihighmobaft` as the key independent variable, as well as (1) a linear time trend, and (2) country fixed effects. To capture (1), we can add `year` as an independent variable, since it is a number and so the coefficient will give us a linear prediction about how the prediction about the top tax rate changes as each year passes (keeping other variables fixed. To capture (2) we can also add `country` like we would as another independent variable, since by default `smf.ols` will convert string variables into a set of separate dummy variables for the different possible values it can take on.

**Question 3.1. Use `smf.ols` to replicate the regression in column 1 of table 1. Make sure the coefficient on `wwihighmobaft` matches what is in the table in the paper. (Side note: the standard errors will be different for reasons we aren't covering in class). Interpret this coefficient.**

In [ ]:
# Code for 3.1

*Words for 3.1*

Columns 2 and 3 add some control variables. To help translate the table from the paper to the variable names:
- "GDP PER CAPITA" is `gdppcp`
- "LEFT SEAT SHARE is `leftseatshp`
- "MALE UNIVERSAL SUFFRAGE" is `munsuff`
- "REVENUE TO GDP" is `ratiop`

To avoid being repetitive, let's skip straight to the *third* column.

**Question 3.2. Use `smf.ols` to replicate the third column of Table 1.**

In [ ]:
# Code for 3.2

To replicate column 4, we need to add *year fixed effects*. I.e., rather than including year as a single independent variable (assuming the effect of going up one year is the same every year), we want to create separate dummy variables for each year of the data. Recall we can do this by adding a `C(varname)` in the `smf.ols` formula.



**Question 3.3. Use `smf.ols` to replicate column 4 of Table 1.**

In [ ]:
# Code for 3.3

The previous analysis just makes a broad comparison between the time before and after WWI for participants (compared to nonparticipants).

Table 2 takes a more fine-grained approach, asking if the top tax rate goes up in individual years of high war mobilization. This analysis is done on a wider time window, from 1850 to 1970.


In [ ]:
all_long = ss[(ss['year'] >= 1850) & (ss['year'] <= 1970)]
all_long

Let's look at how many countries had 2% of the population mobilized for a war effort over the time window. The `himobpopyear2p` variable is equal to 1 when this is true or 0 otherwise, so the yearly average tells us the proportion of countries highly mobilized. (There are some quirks here but the lineplot gives a sense of the general trend.)

In [ ]:
sns.lineplot(x='year', y='himobpopyear2p', data=all_long, ci=None)

Not surprising, we see the biggest spikes around WWI and WWII.

To measure changes in the top tax rate, we include the prior year tax rate `topratepl1`. So, the coefficient on `himobpopyear2p` tells us "keeping fixed the previous year top tax rate, how does having high moblization increase our prediction for the current year top tax rate." That is, how much of an increase over last years top tax rate do we expect with mobilation?

**Question 3.4. Replicate the regression in column 1 of Table 3. Remember to include `year` and `country` as independent variables to capture the linear time trend and country fixed effects.**

In [ ]:
# Code for 3.4

**Question 3.5 [Optional] Replicate the remaining columns of table 3. Hint: to get the decade fixed effects, create a string version of the `decadec` variable.**

## Part 4: Wrapping up
.
**Question 4.1. Run a regression with a different specification than those in the paper. For example, you could add some different control variables, use a different time window, or exclude one or more countries. Explain why you chose this alternative, and interpret the results.**

**Question 4.2. If you had the time/ability to collect additional data to extend this analysis, what would you do? What would you expect to find?**